بازی سودوکو با پای‌گیم 🎮
این نوت‌بوک یک بازی سودوکو را با استفاده از زبان پایتون و کتابخانه‌ی پای‌گیم پیاده‌سازی می‌کند. این پروژه شامل بخش‌های زیر است:

یک تولیدکننده پازل سودوکو

یک حل‌کننده پازل با استفاده از الگوریتم بازگشتی (Backtracking)

یک رابط گرافیکی ساخته‌شده با پای‌گیم برای بازی کردن توسط کاربر

نصب کتابخانه پای‌گیم
برای اجرای بازی باید کتابخانه‌ی pygame را نصب کنید. کافی است این سلول را اجرا کنید:

In [ ]:
!pip install pygame


وارد کردن کتابخانه‌ها و ماژول‌ها
در این بخش، کتابخانه‌های مورد نیاز را وارد می‌کنیم:

In [16]:
import pygame
import sys
from typing import List, Tuple, Optional
import random


pygame: برای ساخت رابط گرافیکی بازی

sys: برای مدیریت برخی عملکردهای سیستم (در اینجا استفاده خاصی نشده اما معمولا همراه با پای‌گیم استفاده می‌شود)

typing: برای نوشتن نوع‌داده‌ها در توابع (Type Hinting)

random: برای تولید اعداد تصادفی در ساخت پازل

مقداردهی اولیه پای‌گیم
قبل از استفاده از قابلیت‌های پای‌گیم، باید آن را مقداردهی اولیه کنیم:

In [ ]:
pygame.init()

# این خط، کتابخانه‌ی پای‌گیم را آماده‌ی استفاده می‌کند.



کلاس SudokuBoard
این کلاس منطق اصلی بازی سودوکو را مدیریت می‌کند:

تولید صفحه بازی

بررسی معتبر بودن حرکات بازیکن

حل پازل با الگوریتم بازگشتی (Backtracking)

تولید پازل با حذف کردن برخی اعداد برای ایجاد چالش

In [18]:
class SudokuBoard:
    def __init__(self):
        # ایجاد یک ماتریس 9x9 پر از صفر (خانه‌های خالی)
        self.board = [[0 for _ in range(9)] for _ in range(9)]
        # نگهداری موقعیت سلول‌های ثابت (خانه‌هایی که بازیکن نمی‌تواند تغییر دهد)
        self.fixed_cells = set()
        # تولید یک پازل جدید در هنگام ساخت نمونه کلاس
        self.generate_puzzle()
        
    def is_valid_placement(self, row: int, col: int, num: int) -> bool:
        # بررسی اینکه عدد num در ردیف row وجود نداشته باشد
        if num in self.board[row]:
            return False
        # بررسی اینکه عدد num در ستون col وجود نداشته باشد
        if num in [self.board[i][col] for i in range(9)]:
            return False
        # پیدا کردن محدوده 3x3 که سلول در آن قرار دارد
        box_row, box_col = 3 * (row // 3), 3 * (col // 3)
        # بررسی اینکه عدد num در جعبه 3x3 وجود نداشته باشد
        for i in range(box_row, box_row + 3):
            for j in range(box_col, box_col + 3):
                if self.board[i][j] == num:
                    return False
        return True
    
    def find_empty_cell(self) -> Optional[Tuple[int, int]]:
        # جستجو برای اولین سلول خالی (مقدار صفر)
        for i in range(9):
            for j in range(9):
                if self.board[i][j] == 0:
                    return (i, j)
        return None  # اگر سلولی خالی نبود، None برمی‌گرداند
    
    def solve(self) -> bool:
        # حل پازل با الگوریتم بازگشتی Backtracking
        empty = self.find_empty_cell()
        if not empty:
            return True  # اگر هیچ خانه خالی نبود، حل شده است
        row, col = empty
        for num in range(1, 10):
            if self.is_valid_placement(row, col, num):
                self.board[row][col] = num
                if self.solve():
                    return True
                # اگر انتخاب عدد موفق نبود، مقدار را صفر (خالی) می‌کند و ادامه می‌دهد
                self.board[row][col] = 0
        return False  # اگر هیچ عددی قابل قرار دادن نبود، False برمی‌گرداند
        
    def is_board_complete_and_valid(self) -> bool:
        # بررسی کامل بودن و معتبر بودن صفحه بازی
        for i in range(9):
            for j in range(9):
                num = self.board[i][j]
                if num == 0:
                    return False  # اگر خانه خالی وجود داشت، کامل نیست
                self.board[i][j] = 0  # موقتا عدد را حذف می‌کند برای بررسی اعتبار
                if not self.is_valid_placement(i, j, num):
                    self.board[i][j] = num  # عدد را دوباره برمی‌گرداند
                    return False  # اگر نامعتبر بود، False برمی‌گرداند
                self.board[i][j] = num
        return True  # اگر همه خانه‌ها معتبر بودند، True برمی‌گرداند
    
    def generate_puzzle(self, difficulty: int = 5) -> None:
        # تولید یک پازل جدید با سطح سختی مشخص
        self.board = [[0 for _ in range(9)] for _ in range(9)]
        self.fixed_cells.clear()
        # پر کردن هر جعبه 3x3 با اعداد تصادفی
        for i in range(0, 9, 3):
            nums = list(range(1, 10))
            random.shuffle(nums)
            for row in range(3):
                for col in range(3):
                    self.board[i + row][i + col] = nums[row * 3 + col]
                    self.fixed_cells.add((i + row, i + col))
        # حل کامل پازل برای داشتن یک بازی کامل
        self.solve()
        # حذف تعدادی از خانه‌ها بر اساس سختی برای ایجاد چالش
        cells = [(i, j) for i in range(9) for j in range(9)]
        random.shuffle(cells)
        for i, j in cells[:difficulty]:
            if (i, j) not in self.fixed_cells:
                self.board[i][j] = 0
    
    def is_cell_fixed(self, row: int, col: int) -> bool:
        # بررسی اینکه آیا سلول جزء خانه‌های ثابت است یا نه
        return (row, col) in self.fixed_cells


کلاس SudokuGUI
این کلاس مسئول نمایش گرافیکی بازی و تعامل با کاربر است:

رسم صفحه بازی

مدیریت کلیک‌های ماوس و فشردن کلیدهای کیبورد

کنترل بازی و نمایش پیام برنده شدن

In [19]:
class SudokuGUI:
    def __init__(self):
        self.WINDOW_SIZE = 540  # اندازه پنجره بازی (پیکسل)
        self.GRID_SIZE = 9      # اندازه جدول 9x9
        self.CELL_SIZE = self.WINDOW_SIZE // self.GRID_SIZE  # اندازه هر خانه
        
        self.game_won = False   # وضعیت برنده شدن بازی
        
        # تعریف رنگ‌ها با فرمت RGB
        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        self.GRAY = (200, 200, 200)
        self.BLUE = (0, 0, 255)
        self.RED = (255, 0, 0)
        
        # ایجاد پنجره بازی
        self.screen = pygame.display.set_mode((self.WINDOW_SIZE, self.WINDOW_SIZE))
        pygame.display.set_caption("بازی سودوکو")
        
        # فونت برای نوشتن اعداد و پیام‌ها
        self.font = pygame.font.Font(None, 40)
        
        self.selected_cell = None  # خانه انتخاب‌شده توسط کاربر
        self.board = SudokuBoard() # نمونه‌ای از کلاس منطق بازی
    
    def draw_grid(self):
        # رسم جدول 9x9 بازی
        for i in range(self.GRID_SIZE):
            for j in range(self.GRID_SIZE):
                x = j * self.CELL_SIZE
                y = i * self.CELL_SIZE
                
                # رنگ پس‌زمینه سلول؛ خاکستری اگر انتخاب شده باشد، سفید در غیر این صورت
                color = self.GRAY if (i, j) == self.selected_cell else self.WHITE
                pygame.draw.rect(self.screen, color, (x, y, self.CELL_SIZE, self.CELL_SIZE))
                
                # رسم خطوط سلول‌ها به رنگ مشکی
                pygame.draw.rect(self.screen, self.BLACK, (x, y, self.CELL_SIZE, self.CELL_SIZE), 1)
                
                # نمایش عدد سلول اگر صفر نباشد
                if self.board.board[i][j] != 0:
                    num = str(self.board.board[i][j])
                    # رنگ عدد: مشکی اگر ثابت باشد، آبی اگر توسط کاربر وارد شده باشد
                    color = self.BLACK if self.board.is_cell_fixed(i, j) else self.BLUE
                    text = self.font.render(num, True, color)
                    text_rect = text.get_rect(center=(x + self.CELL_SIZE//2, y + self.CELL_SIZE//2))
                    self.screen.blit(text, text_rect)
        
        # رسم خطوط ضخیم‌تر برای جدا کردن جعبه‌های 3x3
        for i in range(0, self.WINDOW_SIZE, self.CELL_SIZE * 3):
            pygame.draw.line(self.screen, self.BLACK, (i, 0), (i, self.WINDOW_SIZE), 3)
            pygame.draw.line(self.screen, self.BLACK, (0, i), (self.WINDOW_SIZE, i), 3)
    
    def handle_click(self, pos: Tuple[int, int]) -> None:
        # مدیریت کلیک ماوس: تعیین سلول انتخاب شده
        x, y = pos
        row = y // self.CELL_SIZE
        col = x // self.CELL_SIZE
        if 0 <= row < 9 and 0 <= col < 9:
            # فقط سلول‌هایی که ثابت نیستند می‌توانند انتخاب شوند
            if not self.board.is_cell_fixed(row, col):
                self.selected_cell = (row, col)
    
    def handle_key(self, key: int) -> None:
        # مدیریت فشردن کلیدها برای وارد کردن اعداد یا پاک کردن سلول
        if self.selected_cell is None:
            return
        row, col = self.selected_cell
        
        if key == pygame.K_BACKSPACE or key == pygame.K_DELETE:
            # پاک کردن عدد سلول انتخاب شده
            self.board.board[row][col] = 0
        elif pygame.K_1 <= key <= pygame.K_9:
            # تبدیل کلید به عدد صحیح
            num = key - pygame.K_0
            # اگر قرار دادن عدد معتبر باشد، در جدول قرار می‌گیرد
            if self.board.is_valid_placement(row, col, num):
                self.board.board[row][col] = num
    
    def display_win_message(self):
        # نمایش پیام برنده شدن روی صفحه
        win_text = self.font.render("You Win!", True, self.RED)
        rect = win_text.get_rect(center=(self.WINDOW_SIZE // 2, self.WINDOW_SIZE // 2))
        self.screen.blit(win_text, rect)
    
    def run(self):
        # حلقه اصلی اجرای بازی
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False  # خروج از بازی
                elif not self.game_won:
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        self.handle_click(event.pos)
                    elif event.type == pygame.KEYDOWN:
                        self.handle_key(event.key)
            
            self.screen.fill(self.WHITE)  # پاک کردن صفحه
            self.draw_grid()              # رسم جدول
            
            # بررسی شرایط برنده شدن بازی
            if not self.game_won and self.board.is_board_complete_and_valid():
                self.game_won = True
            
            # نمایش پیام برنده شدن اگر بازی تمام شده باشد
            if self.game_won:
                self.display_win_message()
            
            pygame.display.flip()  # به‌روزرسانی صفحه


اجرای بازی
برای شروع بازی کافی است این بخش را اجرا کنید تا پنجره بازی باز شود و بتوانید بازی کنید:

In [ ]:
if __name__ == "__main__":
    game = SudokuGUI()
    game.run()
